# Final model evaluation inference

**THIS IS EXPERIMENTAL**. Doing minor modifications on this notebook, you could do inference on new data!

Unfortunately, my focus has not been on this (and it shows).

But, if you need some help, let me know :)

## Import libraries

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import time
from sklearn.preprocessing import StandardScaler
import pickle 
import re
import tqdm

import nltk
from nltk.corpus import stopwords # Not supporting Polish :/
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer # Not supporting Polish :/
from pystempel import Stemmer

from sklearn.impute import SimpleImputer
from catboost import CatBoostRegressor

import sys, os
sys.path.append(os.getcwd()+ "/../")
from src.data_preprocessing import DataPreprocessing
from src.compute_metrics import Metrics

## Load your data

In [18]:
# assert that you have the correct version of the columns
your_data_path = "../data/real_estate_ads_2022_10.csv" # Should be a csv file.
# It should contain 'price' and 'price_per_m' features (set them to dummy)
# this should be corrected in the data preprocessing step
# but it is not yet supported

dp = DataPreprocessing(df_path = your_data_path,
                        train_indices_path="../data/train_indices.npy", 
                        test_indices_path="../data/test_indices.npy",
                        get_params_from_params=True,
                        get_tfidf_embeddings_flag=False,
                        get_bert_embeddings_flag=False, # IMPORTANT: WITHOUT BERT EMBEDDINGS
                        get_textual_features_flag=True,
                        transform_time_features_flag=True,
                        transform_cyclic_features_flag=True,
                        drop_unnecessary_columns_at_end_flag=False)

with open("../data/tfidf_vectorizer_description.pkl", "rb") as f:
    dp.tfidf_vectorizer_description = pickle.load(f)
with open("../data/tfidf_vectorizer_title.pkl", "rb") as f:
    dp.tfidf_vectorizer_title = pickle.load(f)

In [19]:
nltk.download('punkt')
nltk.download('stopwords')

with open("../data/polish.stopwords.txt", "r") as file:
    polish_stopwords = file.read().splitlines()

stop_words = set(polish_stopwords)
stemmer = Stemmer.default()

def preprocess_sentence(sentence):

    sentence = str(sentence)

    # Convert to lowercase
    sentence = sentence.lower()
    
    # Remove special characters and numbers
    # (although numbers may be useful in some cases)
    # (for time constraints, I don't experiment on this)
    sentence = re.sub(r'\d+|\W+', ' ', sentence)
    
    # Tokenize the sentence
    words = word_tokenize(sentence)
    
    # Remove stopwords and apply stemming
    processed_words = [stemmer(word) for word in words if word not in stop_words]
    
    return processed_words

def clean_html_tags(text):
    # Remove HTML tags using regex
    clean_text = re.sub(r'<[^>]+>', '', text)
    
    # Replace multiple consecutive spaces or newlines with a single space
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    
    return clean_text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sergi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sergi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Loading: 100%|██████████| 2225192/2225192 [00:00<00:00, 2471239.89bytes/s]


### Preprocess descriptions and titles
This takes a bit (less than 5min).
This id due to the preprocessing of descriptions. Optimizations should be considered.

In [20]:
tqdm.tqdm.pandas()
preprocessed_descriptions = dp.X["description"].progress_apply(clean_html_tags)
preprocessed_descriptions = preprocessed_descriptions.progress_apply(preprocess_sentence)
preprocessed_titles = dp.X["title"].progress_apply(preprocess_sentence)

100%|██████████| 73645/73645 [00:07<00:00, 9592.15it/s] 


In [21]:
preprocessed_descriptions = preprocessed_descriptions.apply(lambda x: np.str_(x))
preprocessed_titles = preprocessed_titles.apply(lambda x: np.str_(x))

In [22]:
preprocessed_descriptions

0        ['zapraszać', 'zakup', 'mieszkać', 'zlokalizow...
1        ['polecać', 'sprzedaż', 'mieszkać', 'kawalerić...
2        ['przestronny', 'mieszkać', 'pokojowy', 'podda...
3        ['serdeczn', 'zapraszać', 'zapoznanie', 'ofert...
4        ['polecać', 'sprzedaż', 'mieszkać', 'komórka',...
                               ...                        
73640    ['drogi', 'państwo', 'zapraszać', 'zapoznanie'...
73641    ['polecać', 'sprzedaż', 'mieszkać', 'pokojowy'...
73642    ['sprzedaż', 'przestronny', 'mieszkać', 'położ...
73643    ['kontae', 'sprawo', 'oferta', 'katarzyć', 'pi...
73644    ['sprzedać', 'pokojowy', 'zadbany', 'mieszkać'...
Name: description, Length: 73645, dtype: object

In [23]:
tfidf_title_features = dp.tfidf_vectorizer_title.transform(preprocessed_titles)
tfidf_description_features = dp.tfidf_vectorizer_description.transform(preprocessed_descriptions)

## Scale
### Import the scalers

In [24]:

with open("../data/reduction_models/pca_tfidf_descriptions.pkl", "rb") as f:
    pca_tfidf_descriptions = pickle.load(f)

with open("../data/reduction_models/pca_tfidf_titles.pkl", "rb") as f:
    pca_tfidf_titles = pickle.load(f)

with open("../data/reduction_models/pls2_tfidf_descriptions.pkl", "rb") as f:
    pls2_tfidf_descriptions = pickle.load(f)

with open("../data/reduction_models/pls2_tfidf_titles.pkl", "rb") as f:
    pls2_tfidf_titles = pickle.load(f)

### Scale

In [25]:
pca_tfidf_titles_features = pca_tfidf_titles.transform(tfidf_title_features.toarray())
pca_tfidf_titles_columns = ["pca_tfidf_titles_" + str(i) for i in range(pca_tfidf_titles_features.shape[1])]
pca_tfidf_titles_features = pd.DataFrame(pca_tfidf_titles_features, columns=pca_tfidf_titles_columns)

pca_tfidf_descriptions_features = pca_tfidf_descriptions.transform(tfidf_description_features.toarray())
pca_tfidf_descriptions_columns = ["pca_tfidf_descriptions_" + str(i) for i in range(pca_tfidf_descriptions_features.shape[1])]
pca_tfidf_descriptions_features = pd.DataFrame(pca_tfidf_descriptions_features, columns=pca_tfidf_descriptions_columns)

pls2_tfidf_titles_features = pls2_tfidf_titles.transform(tfidf_title_features.toarray())
pls2_tfidf_titles_columns = ["pls2_tfidf_titles_" + str(i) for i in range(pls2_tfidf_titles_features.shape[1])]
pls2_tfidf_titles_features = pd.DataFrame(pls2_tfidf_titles_features, columns=pls2_tfidf_titles_columns)

pls2_tfidf_descriptions_features = pls2_tfidf_descriptions.transform(tfidf_description_features.toarray())
pls2_tfidf_descriptions_columns = ["pls2_tfidf_descriptions_" + str(i) for i in range(pls2_tfidf_descriptions_features.shape[1])]
pls2_tfidf_descriptions_features = pd.DataFrame(pls2_tfidf_descriptions_features, columns=pls2_tfidf_descriptions_columns)

### Append features to model

In [26]:
dp.X = pd.concat([dp.X, pca_tfidf_titles_features, pca_tfidf_descriptions_features, pls2_tfidf_titles_features, pls2_tfidf_descriptions_features], axis=1)

## Load final model

In [27]:
import pickle as pkl

with open("../data/models/final_model_catboost.pkl", "rb") as f:
    model = pkl.load(f)

In [28]:
dp.X

,district_lon,district_lat,title,description,no_rooms,m,map_lon,map_lat,market_secondary,param_building_floors_num,...,pca_tfidf_titles_4,pca_tfidf_descriptions_0,pca_tfidf_descriptions_1,pca_tfidf_descriptions_2,pca_tfidf_descriptions_3,pca_tfidf_descriptions_4,pls2_tfidf_titles_0,pls2_tfidf_titles_1,pls2_tfidf_descriptions_0,pls2_tfidf_descriptions_1
0,16.90502,52.41180,Mieszkanie z potencjałem. Rynek Jeżycki.,<p>Zapraszam do zakupu mieszkania zlokalizowan...,2,51.00,16.896592,52.410150,1,3.0,...,-0.025490,-0.088239,-0.165477,-0.349479,0.010310,-0.055719,-0.186961,0.070337,1.970348,3.060868
1,16.87466,52.41572,Okazja Jeżyce!!!!!,<p>Polecam na sprzedaż mieszkanie kawalerka o ...,1,26.00,16.904410,52.411919,1,4.0,...,-0.035011,-0.060239,-0.149004,-0.376304,0.010743,-0.143311,-1.322833,1.053056,4.320092,2.411582
2,16.90502,52.41180,"**Okazja * 60,1m2 * 3 pokoje * poddasze * Jeży...",<p> **Przestronne mieszkanie 3 pokojowe * na ...,3,60.10,16.899310,52.413357,1,4.0,...,-0.035011,-0.078891,-0.137050,-0.342215,0.022760,0.026807,-1.322833,1.053056,0.707916,2.390137
3,16.87466,52.41572,"Mieszkanie 2-pokojowe, Ogrody, Kampus Uam",<p>Serdecznie zapraszam do zapoznania się z of...,2,47.00,16.883608,52.410662,1,4.0,...,-0.025490,-0.109846,-0.149579,-0.326880,-0.012918,0.168087,-0.186961,0.070337,0.342554,0.049286
4,16.87466,52.41572,Jeżyce 2 pokoje,<p>Polecam na sprzedaż mieszkanie z komórka lo...,2,44.00,16.890796,52.412308,1,4.0,...,-0.025490,-0.076983,-0.140057,-0.360769,0.028240,-0.068605,-0.186961,0.070337,2.560499,2.602684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73640,16.94080,52.38356,Poznań - Rataje idealny układ pomieszczeń.,"<p>Drodzy Państwo,</p>\r\n<p>Zapraszam do zapo...",3,47.00,16.955818,52.389898,1,4.0,...,-0.025490,-0.087246,-0.137394,-0.347674,0.005130,0.102709,-0.186961,0.070337,1.180522,0.610046
73641,16.94080,52.38356,2 pokojowe z balkonem RATAJE os. Piastowskie,<p>Polecam na sprzedaż mieszkanie 2 pokojowe u...,2,43.70,16.947615,52.392968,1,4.0,...,-0.031914,-0.059395,-0.158925,-0.380062,-0.012824,-0.050761,0.386344,0.118513,3.297869,0.525983
73642,16.90590,52.36596,"Inwestycyjne 2 niezależne pokoje, do wykończenia.",<p>Na sprzedaż przestronne mieszkanie położone...,2,45.32,16.904862,52.369679,1,4.0,...,-0.025490,-0.086130,-0.151370,-0.362075,-0.012937,0.085552,-0.186961,0.070337,2.820646,-2.864064
73643,16.94080,52.38356,Przestronna kawalerka z dużym tarasem Chartowo...,<p>Kontakt w sprawie oferty - Katarzyna Piotro...,1,36.30,16.980725,52.389168,1,18.0,...,-0.025490,-0.100513,-0.137185,-0.317533,0.011127,0.139181,-0.186961,0.070337,0.427930,-0.063564


In [29]:
dp.get_train_test_split(dp.X)

(       district_lon  district_lat  \
 34086      16.94658      52.45695   
 41636      16.96521      52.38044   
 6943       16.92186      52.41220   
 23851      16.91912      52.45528   
 18563      16.87200      52.39591   
 ...             ...           ...   
 37194      16.94080      52.38356   
 6265       16.92186      52.41220   
 54886      16.96358      52.39865   
 860        16.94658      52.45695   
 15795      16.88339      52.45718   
 
                                                    title  \
 34086    3 pokojowe mieszkanie 49,57m2 - zielona okolica   
 41636           Mieszkanie blisko centrum - lok. B/0/M04   
 6943   Dwupokojowe w dobrym stanie w dogodnej lokaliz...   
 23851                  Kawalerka na poznańskim Piątkowie   
 18563   2006 dwupokojowe z osobną kuchnią i komórką lok.   
 ...                                                  ...   
 37194  Mieszkaj z ogródkiem M2 30,29m2-Wagrowska OSTA...   
 6265           Lokalizacja jak marzenie, bardzo zadba

## Compute predictions

In [38]:
try:
    dp.X = dp.X.drop(columns=["title", "description"])
except:
    pass

y_predict = pd.DataFrame(model.predict(dp.X), columns=["price"])
y_predict = dp.backward_transform_price_per_m(y_predict)

### See the results

In [39]:
y_predict

,price,price_per_m
0,-1.226904,5441.036976
1,-0.016836,7965.342312
2,-1.300241,5308.119257
3,-0.332357,7244.864411
4,-0.048981,7889.849186
...,...,...
73640,-0.078141,7821.779836
73641,-0.483009,6916.750285
73642,-1.020268,5827.436699
73643,-0.448614,6990.769950
